# Tratamento Inicial de Dados do ENEM 2022

## Objetivo

Como o dataset do enem é bem grande, seria interessante reduzir seu número de parâmetros, para facilitar futuras análises.

## Importanto Dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv", encoding="iso-8859-1", delimiter=";")
df.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210057943671,2022,14,M,2,2,1,1,2,1,...,B,A,A,A,A,A,A,A,A,A
1,210057516120,2022,14,M,2,1,1,1,16,1,...,E,E,B,E,B,B,E,B,E,B
2,210057280536,2022,5,F,1,2,1,1,2,1,...,A,A,A,A,A,A,C,A,A,B
3,210055724397,2022,6,M,1,3,1,1,2,1,...,B,A,A,C,A,A,C,B,B,B
4,210055097896,2022,4,M,0,3,1,1,1,1,...,A,A,A,A,A,A,B,A,A,A


Como podemos ver, a tabela tem 76 colunas. Muitas delas não são relevantes para a nossa análise (ex: vetor com as questões que o inscrito marcou). Outras possues um valor marcado com algo quando seria mais interessante se fosse NA (ex: valor 1 no tipo de escola quer dizer que não respondeu).<br><br>
A partir de agora, iremos eliminar aquelas colunas julgadas como não relevantes, e transformar os dados caso seja necessário.<br><br>
Tratamentos mais avançados ou bem específicos da análise que for feita serão feitos num código a parte.

## Eliminando Colunas Desnecessárias

As colunas podem ser vistas no diretório DICIONÁRIO.

In [3]:
# Todos os dados são do ENEM 2022
df.drop("NU_ANO", axis=1, inplace=True)

In [4]:
# O Estado Civil não é considerado relevante para as análises que serão feitas. 
# Ele não dá informações relevantes sobre o desempenho do candidato da prova, e nem possui relação com cotas
# Ano de conclusão de ensino médio também (o que é relevante é se está fazendo ou não)
df.drop("TP_ESTADO_CIVIL", axis=1, inplace=True)
df.drop("TP_ANO_CONCLUIU", axis=1, inplace=True)

In [5]:
# Quanto à região da escola, algumas informações podem ser relevantes
# Para esse trabalho, será considerado que apenas a UF da escola é relevante. O município é uma informação muito específica, e vai gerar um número muito alto de classes.
# Informações como localização da escola e estado de funcionamento podem ser bem legais, entretanto, para limitar o número de parâmetros, será considerado que saber se a escola
# é pública ou privada e em qual UF ela fica são informações suficientes
df.drop("CO_MUNICIPIO_ESC", axis=1, inplace=True)
df.drop("NO_MUNICIPIO_ESC", axis=1, inplace=True)
df.drop("TP_DEPENDENCIA_ADM_ESC", axis=1, inplace=True)
df.drop("CO_UF_ESC", axis=1, inplace=True)
df.drop("TP_LOCALIZACAO_ESC", axis=1, inplace=True)
df.drop("TP_SIT_FUNC_ESC", axis=1, inplace=True)

In [6]:
# Todos os dados sobre o local da prova serão considerados irrelevantes
df.drop("CO_MUNICIPIO_PROVA", axis=1, inplace=True)
df.drop("NO_MUNICIPIO_PROVA", axis=1, inplace=True)
df.drop("CO_UF_PROVA", axis=1, inplace=True)
df.drop("SG_UF_PROVA", axis=1, inplace=True)

In [7]:
# Sobre dados da prova objetivo, serão utilizados dados sobre a presença nas provas (pode ser útil mais pra frente)
# Dados sobre qual o código da prova que o aluno fez serão mantidos para análises, mas provavelmente serão retirados para o modelo de previsão
# O vetor de respostas será eliminado, pois não iremos analisar quais questões cada aluno errou (motivo análogo para eliminr o vetor de gabarito)
# A língua estrangeira será eliminada também, pois ela é considerada irrelevante para as análises que serão feitas
df.drop("TX_RESPOSTAS_CN", axis=1, inplace=True)
df.drop("TX_RESPOSTAS_CH", axis=1, inplace=True)
df.drop("TX_RESPOSTAS_LC", axis=1, inplace=True)
df.drop("TX_RESPOSTAS_MT", axis=1, inplace=True)

df.drop("TP_LINGUA", axis=1, inplace=True)

df.drop("TX_GABARITO_CN", axis=1, inplace=True)
df.drop("TX_GABARITO_CH", axis=1, inplace=True)
df.drop("TX_GABARITO_LC", axis=1, inplace=True)
df.drop("TX_GABARITO_MT", axis=1, inplace=True)

In [8]:
# Sobre os dados da Redação, todos serão mantidos
# Embora provavelmente a nota de cada competência não será usada no modelo de previsão, essas informações podem ser interessantes de serem analisadas separadamente

In [9]:
# Sobre o Formulário Socioeconômico, há várias informações específicas que podem ser interessantes em análises específicas
# Para esse trabalho, a única considerada relevante é sobre a renda familiar (Questão 6). As outras serão eliminadas
eliminar = [f"Q{i:03d}" for i in range(1, 26)]  # as colunas vão de Q001 até Q025
eliminar.remove("Q006")  # questão da renda familiar
for col in eliminar:
    df.drop(col, axis=1, inplace=True)

Após as eliminações, temos um total de 31 colunas.

## Tratamento dos Dados

Nesse documento serão feitos tratamentos mais simples (ex: gerar valores NA). Tratamentos mais complexos (ex: tratamentos que exigem criar novas colunas) serão feitos na etapa de Data Cleaning para o modelo de previsão.

In [10]:
# Células que envolvem algum valor "não informado" serão alteradas para NA
df["TP_COR_RACA"].replace({0: np.nan, 6: np.nan}, inplace=True)
df["TP_NACIONALIDADE"].replace(0, np.nan, inplace=True)
df["TP_ESCOLA"].replace(1, np.nan, inplace=True)

In [11]:
# Como agora só há duas opções de escola (Pública ou Privada), dá para usar só dois valores
# Pública = 1 e Privada = 0 (pode ser interpretado como "elegível à cota?")
df["TP_ESCOLA"].replace({2: 1, 3: 0}, inplace=True)
df["TP_ESCOLA"].value_counts()

TP_ESCOLA
1.0    1105355
0.0     212205
Name: count, dtype: int64

In [12]:
# Quantos valores ainda vamos ter se dropar valores na?
df.dropna().shape[0]

669781

In [13]:
# Deletando valores na
df.dropna(inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 669781 entries, 40 to 3476102
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   NU_INSCRICAO       669781 non-null  int64  
 1   TP_FAIXA_ETARIA    669781 non-null  int64  
 2   TP_SEXO            669781 non-null  object 
 3   TP_COR_RACA        669781 non-null  float64
 4   TP_NACIONALIDADE   669781 non-null  float64
 5   TP_ST_CONCLUSAO    669781 non-null  int64  
 6   TP_ESCOLA          669781 non-null  float64
 7   TP_ENSINO          669781 non-null  float64
 8   IN_TREINEIRO       669781 non-null  int64  
 9   SG_UF_ESC          669781 non-null  object 
 10  TP_PRESENCA_CN     669781 non-null  int64  
 11  TP_PRESENCA_CH     669781 non-null  int64  
 12  TP_PRESENCA_LC     669781 non-null  int64  
 13  TP_PRESENCA_MT     669781 non-null  int64  
 14  CO_PROVA_CN        669781 non-null  float64
 15  CO_PROVA_CH        669781 non-null  float64
 16  CO_PR

## Conclusão

Infelizmente, muitas pessoas não respondem alguns campos (principalmente o de informar se a escola é pública ou privada). Por causa disso, um dataset de mais de 3 milhões de observações foi reduzido para 600 mil observações. Entretanto, ainda é um número considerável para fazer análises.

In [15]:
# Slavando dataset
df.to_csv("enem2022_dataset.csv", index=False)